## Automated Preprocessing - Example Titanic Dataset

In [81]:
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
# from pyod.models.iforest import IForest
# from pyod.models.lof import LOF
from pyod.models.pca import PCA

import sys
sys.path.append('../../')

from pipelines.control import AutoPrepAD


In [82]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

train_with_survived = train.set_index("PassengerId")

train = train.set_index("PassengerId").drop("Survived", axis=1)
test = test.set_index("PassengerId")


train
# print(train)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [83]:
test

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [84]:
df = pd.concat([train, test], axis=0)
df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [85]:
pipeline = AutoPrepAD(
    nominal_columns=["Sex", "Ticket", "Cabin", "Embarked"],
    exclude_columns=["Name"]
                      )
df_transformed = pipeline.preprocess(df)

Transforming Input Dataframe with Pipeline...
No Variance in follow Train Columns:  Index(['Nominal Columns__nominal_processing_inner__Embarked_0'], dtype='object')
Only NaNs in follow Train Columns:  Index([], dtype='object')
Remove columns with zero variance: True
Shape Preprocessed before drop: (1309, 30)
Shape Preprocessed after drop: (1309, 29)

Check NaN Preprocessed: []
Check inf Preprocessed: []


In [86]:
df_train_transformed.head(4)

,Preprocessing Pipeline__Numerical__stat_tukey__Tukey_Total,Preprocessing Pipeline__Numerical__z_mod__MAD_Total,Preprocessing Pipeline__Numerical__pass_cols__numeric__X__X__Pclass,Preprocessing Pipeline__Numerical__pass_cols__numeric__X__X__Age,Preprocessing Pipeline__Numerical__pass_cols__numeric__X__X__SibSp,Preprocessing Pipeline__Numerical__pass_cols__numeric__X__X__Parch,Preprocessing Pipeline__Numerical__pass_cols__numeric__X__X__Fare,Nominal Columns__nominal_processing_inner__Name_0,Nominal Columns__nominal_processing_inner__Name_1,Nominal Columns__nominal_processing_inner__Name_2,Nominal Columns__nominal_processing_inner__Name_3,Nominal Columns__nominal_processing_inner__Name_4,Nominal Columns__nominal_processing_inner__Name_5,Nominal Columns__nominal_processing_inner__Name_6,Nominal Columns__nominal_processing_inner__Name_7,Nominal Columns__nominal_processing_inner__Name_8,Nominal Columns__nominal_processing_inner__Name_9,Nominal Columns__nominal_processing_inner__Name_10,Nominal Columns__nominal_processing_inner__Sex_0,Nominal Columns__nominal_processing_inner__Sex_1,Nominal Columns__nominal_processing_inner__Ticket_0,Nominal Columns__nominal_processing_inner__Ticket_1,Nominal Columns__nominal_processing_inner__Ticket_2,Nominal Columns__nominal_processing_inner__Ticket_3,Nominal Columns__nominal_processing_inner__Ticket_4,Nominal Columns__nominal_processing_inner__Ticket_5,Nominal Columns__nominal_processing_inner__Ticket_6,Nominal Columns__nominal_processing_inner__Ticket_7,Nominal Columns__nominal_processing_inner__Ticket_8,Nominal Columns__nominal_processing_inner__Ticket_9,Nominal Columns__nominal_processing_inner__Cabin_0,Nominal Columns__nominal_processing_inner__Cabin_1,Nominal Columns__nominal_processing_inner__Cabin_2,Nominal Columns__nominal_processing_inner__Cabin_3,Nominal Columns__nominal_processing_inner__Cabin_4,Nominal Columns__nominal_processing_inner__Cabin_5,Nominal Columns__nominal_processing_inner__Cabin_6,Nominal Columns__nominal_processing_inner__Cabin_7,Nominal Columns__nominal_processing_inner__Embarked_1,Nominal Columns__nominal_processing_inner__Embarked_2
PassengerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1,0.0,-0.461538,1.0,0.0,-0.308146,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
2,3,3,-2.0,0.769231,1.0,0.0,2.430755,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
3,0,0,0.0,-0.153846,0.0,0.0,-0.279274,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1
4,3,3,-2.0,0.538462,1.0,0.0,1.652999,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1


In [87]:
df_train_transformed = df_transformed.loc[:891,:]
df_test_transformed = df_transformed.loc[892:,:]

df_train_transformed.shape

(891, 29)

In [88]:
df_test_transformed.shape

(418, 29)

In [89]:
df_train_transformed
y_true = train_with_survived["Survived"]

df_test_transformed.shape

(418, 29)

In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df_train_transformed, y_true, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=2000, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred_val = rf_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred_val)
print(f'Validation Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_val, y_pred_val))

# Make predictions on the test set
predictions_test = rf_model.predict(df_test_transformed)

# Create a DataFrame to save the predictions
df_test_transformed['Survived'] = predictions_test


Validation Accuracy: 0.7877094972067039
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       105
           1       0.79      0.66      0.72        74

    accuracy                           0.79       179
   macro avg       0.79      0.77      0.77       179
weighted avg       0.79      0.79      0.78       179



In [91]:
df_test_transformed.reset_index(inplace=True)

In [92]:
# Create a DataFrame to save the predictions with PassengerId and Survived columns
df_test_predictions = pd.DataFrame({
    'PassengerId': df_test_transformed['PassengerId'],
    'Survived': predictions_test
})

# Output the DataFrame with predictions
df_test_predictions.to_csv('submission.csv', index=False)
print('Test predictions saved to submission.csv')

Test predictions saved to test_predictions.csv


In [93]:
# df_train_transformed["Survived"] = train_with_survived["Survived"]
# df_train_transformed